In [ ]:
!pip install web3 sha3 ecpy
!pip install protobuf==4.21.6
import web3
import numpy as np
import requests
import sha3
import json
from ecpy import curves, keys
import eth_abi
import eth_abi.packed
import hexbytes

In [ ]:
pk = 0x   #Private Key

In [ ]:
cv = curves.Curve.get_curve('secp256k1')
pv_key = keys.ECPrivateKey(pk, cv)
pu_key = pv_key.get_public_key()
concat_x_y = pu_key.W.x.to_bytes(32, byteorder='big') + pu_key.W.y.to_bytes(32, byteorder='big')
eth_addr = web3.Web3.keccak(concat_x_y)[-20:].hex()
ch_eth_addr = web3.Web3.to_checksum_address(eth_addr)
ch_eth_addr

In [ ]:
w3 = web3.Web3(web3.HTTPProvider('https://eth-sepolia.g.alchemy.com/v2/jpZvH2isfX1Pppx5VUjXg0pCQK0mllb0'))
w3.is_connected()

In [ ]:
nulladdr = '0x'+'0'*40

In [ ]:
amount = 0.1
jobAddress = w3.to_checksum_address('0x4d6409411Fc18619895A9eD707DBc98ad6708C45')
jobSelector = hexbytes.HexBytes(0xd09de08a) #0xd09de08a selector #0x3459b7ea calldata
useJobOwnerCredits = True
assertResolverSelector = False
maxBaseFeeGwei = 500
rewardPct = 0
fixedReward = 5000
jobMinCvp = 1000000000000000000000
calldataSource = 0
intervalSeconds = 120
resolverAddress =  "0x4d6409411Fc18619895A9eD707DBc98ad6708C45" #"0x3Bc5e58EF60e83219FC71e659023B5a22713915a" #"0x276352c103FB5d66584A70DaC5AB356FA40bD7d7" #nulladdr
resolverCalldata =  hexbytes.HexBytes(0x0ec6af7b) #hexbytes.HexBytes(0x0ec6af7b) #hexbytes.HexBytes(0x00)
preDefinedCalldata = hexbytes.HexBytes(0x00) #hexbytes.HexBytes(0x3459b7ea000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000142b8c6f9cf28ca722d891bb46f83d3896848cb019000000000000000000000000) #0x000000000000000000000000da2abfdce949e4e8158829176e7e8af493565b6d

In [ ]:
apiEndpoint = 'https://api-sepolia.etherscan.io/api?module=contract&action=getabi&address='
agentAddr = '0xc8E864f12c337Bdf6294a3DCeE0E565D2B1B4d90'#The latest Agent address
toCall = apiEndpoint+agentAddr

In [ ]:
agentAbi = json.loads(requests.get(toCall).json()['result'])
agent = w3.eth.contract(abi = agentAbi, address = agentAddr)

In [ ]:
params_ = {
    'params_': (jobAddress, jobSelector, useJobOwnerCredits, assertResolverSelector, maxBaseFeeGwei, rewardPct, fixedReward, jobMinCvp, calldataSource, intervalSeconds),
    'resolver_': (resolverAddress, resolverCalldata),
    'preDefinedCalldata_': preDefinedCalldata
}

In [ ]:
tx = agent.functions.registerJob(**params_).build_transaction(
    {'from': ch_eth_addr,
     'nonce': w3.eth.get_transaction_count(ch_eth_addr),
     'value': int(amount*1e18)}
)

In [ ]:
st = w3.eth.account.sign_transaction(tx, pk)
w3.eth.send_raw_transaction(st.rawTransaction)